# API: Preparation, training and testing of the model on exemplary input


### Idea 
user input his(personality test answers)and the system recommeds his matching Cats 


### Metodology 
since our users don't have any prior preferences towards Cats(USER COLD START problem) we have to start with this:

1. Finding **nearset user**: the system takes user input (personality test answers) then finds the nearsest user (the most similar one) from our DataSet
    - for better pridections : we filterd out users with satisfaction rate below 80%

2. we get **Connceted Cat** of this nearest user (from the training DataSet)

3. based on the **Connceted Cat** the system recommends the nearset cat s we have available on the website


some notes: 

- there isn't any active learning,the system just recommends based on the Dataset (aka. the scale where every user get his recommended cat)  
- due to the DataSet setup, each user is exposed to one and only one Cat (Aka. each user rates one Cat). so we are forced to use "**user_based** collaborative filtering", in Addition satisfaction here is based on live -interaction between the Adopter and the cat for some time, so the satisfaction is a reliable scale 

### Import libraries

In [268]:
import numpy as np
import pandas as pd
import joblib
from sklearn.neighbors import NearestNeighbors

pd.set_option('display.max_columns', 1000)

### Load data

In [174]:
df = pd.read_excel('https://github.com/TechLabs-Berlin/wt21-pet-match/blob/main/data_set/data_cleaned/all_cleaned_num.xlsx?raw=true')
df = df.set_index('id').drop(columns = ['breed', 'easily_nervous', 'talkative', 'active_imagination', 'thorough job'])
#df.describe()

### filter out the unstisfied users 

In [262]:
df['satisf_level'].value_counts(normalize = True)

1.00    0.582408
0.95    0.147403
0.90    0.085860
0.85    0.075053
0.80    0.039928
0.00    0.024917
0.75    0.018613
0.70    0.011408
0.65    0.003903
0.60    0.003603
0.55    0.002402
0.40    0.001501
0.50    0.001501
0.35    0.000600
0.45    0.000600
0.25    0.000300
Name: satisf_level, dtype: float64

- nearly 58% are fully staisfied (satisfaction = 1), while (~10%) are blew 0.8 satisfaction.
- since the recommender needs the users and Adopters to be high satisfied we choose to remove subjects with satisfaction less than 0.8 

In [263]:
df_happy = df.loc[df["satisf_level"] > 0.8].drop(columns=["satisf_level"]).reset_index(drop=True)

In [264]:
# setting up a subset for the users_only features 
df_adopters = df_happy.loc[:, "adopter_gender":"efficient"]
df_adopters['adopter_age'].replace({1: 23, 2: 30, 3: 40, 4: 50, 5: 60, 6: 70}, inplace = True)

/home/omaistat/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:6619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [266]:
# a subset with cats' features
df_cats = df_happy.loc[:, "cat_age":"fearful"] 
df_cats

,cat_age,cat_gender,needs_outdoor,medical_conditions,behavioural_problems,cat_weight,likes_to_explore,playful,vocal,picked_up,timid,aggressive,adapts_quickly,prefers_alone,likes_stroke,tolerant_handled,friendly,fearful
0,6,1,2,2,1,3,4,4,3,2,3,1,3,2,4,3,4,5
1,2,2,2,1,1,3,4,4,1,4,3,3,2,3,4,3,5,2
2,5,1,1,1,1,4,5,3,4,2,2,3,4,3,3,2,3,3
3,4,2,2,1,1,4,3,4,3,5,4,2,2,2,5,4,4,4
4,3,2,2,1,1,5,4,3,5,2,3,1,4,3,4,3,4,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2962,12,2,1,1,1,3,5,5,3,4,4,1,4,2,5,4,5,2
2963,7,1,2,1,1,3,5,4,3,4,3,1,4,1,5,5,4,2
2964,4,1,1,1,1,3,5,5,5,5,1,1,5,2,5,5,5,1
2965,1,1,1,1,1,3,4,5,3,5,4,3,5,1,5,5,5,4


## Finding nearest user 

### Test users

In [235]:
test_user = [[1,20,4,3,2,5,4,3,4,5,3,4,5,2,3]]

user_8 = [[2, 30, 0, 1, 4, 4, 4, 3, 5, 4, 2, 5, 5, 3, 5]]

# NearestNeighbor Users

In [237]:
neigh_users = NearestNeighbors(n_neighbors=1, metric = 'correlation')

In [238]:
neigh_users.fit(df_adopters)

NearestNeighbors(metric='correlation', n_neighbors=1)

In [275]:
nearest_user = int(neigh_users.kneighbors(user_8, return_distance=False))
nearest_user


/home/omaistat/anaconda3/lib/python3.8/site-packages/sklearn/base.py:445: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


8

# Nearest Neighbor Cats

In [246]:
neigh_cats = NearestNeighbors(n_neighbors=10, metric = 'correlation')

In [247]:
neigh_cats.fit(df_cats)

NearestNeighbors(metric='correlation', n_neighbors=10)

In [285]:
result = pd.DataFrame(neigh_cats.kneighbors(pd.DataFrame(df_cats.iloc[nearest_user]).transpose(), return_distance=False)).transpose()
result.columns = ['catID']
result

,catID
0,8
1,2241
2,425
3,1237
4,2734
5,1767
6,2496
7,141
8,2188
9,981


In [284]:
result_list = []
i = 1
for cat_id in result['catID']:
                case = {'catOrder': i, 'catID': cat_id}
                result_list.append(case.copy())
                i += 1
result_list

[{'catOrder': 1, 'catID': 8},
 {'catOrder': 2, 'catID': 2241},
 {'catOrder': 3, 'catID': 425},
 {'catOrder': 4, 'catID': 1237},
 {'catOrder': 5, 'catID': 2734},
 {'catOrder': 6, 'catID': 1767},
 {'catOrder': 7, 'catID': 2496},
 {'catOrder': 8, 'catID': 141},
 {'catOrder': 9, 'catID': 2188},
 {'catOrder': 10, 'catID': 981}]

# Dumping the adopters model

In [271]:
joblib.dump(neigh_users, 'model_users.pkl')
print("Users Model dumped!")
model_user = joblib.load('model_users.pkl')

model_columns = list(df_adopters.columns)
joblib.dump(model_columns, 'model_users_columns.pkl')
print("User model columns dumped!")

Users Model dumped!
User model columns dumped!


# Dumping the cats model

In [272]:
joblib.dump(neigh_cats, 'model_cats.pkl')
print("Cats Model dumped!")
#model_cats = joblib.load('model_cats.pkl')

model_columns = list(df_cats.columns)
joblib.dump(model_columns, 'model_cats_columns.pkl')
print("Cats model columns dumped!")

Cats Model dumped!
Cats model columns dumped!


# Check later

### 